In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.collection.mutable.ListBuffer
import scala.io.Source
import scala.util.control._
implicit def bool2int(b:Boolean) = if (b) 1 else 0

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-OGTOHVK:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1605732569062)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.collection.mutable.ListBuffer
import scala.io.Source
bool2int: (b: Boolean)Int


In [41]:
// parameters 
val data = "../data/T10I4D100K.dat"
val support = 1000
val k = 1

// frequent singelton
val singletons = Array.fill(100000)(0)
Source.fromFile(data).getLines().map(x => x.split(" ").map(item => item.toInt)).foreach(_.foreach(singletons(_) += 1))

val L1 = singletons.zipWithIndex.filter(x => x._1 >= support).map(x => Set(x._2)).toSet

data: String = ../data/T10I4D100K.dat
support: Int = 1000
k: Int = 1
singletons: Array[Int] = Array(594, 1535, 673, 531, 1394, 1094, 2149, 997, 3090, 0, 1351, 525, 3415, 35, 197, 458, 150, 1683, 813, 121, 40, 2666, 397, 128, 191, 1395, 527, 2165, 1454, 171, 0, 1666, 4248, 1460, 56, 1984, 528, 1249, 2402, 4258, 457, 1353, 119, 1721, 903, 1728, 543, 175, 2472, 137, 0, 1612, 1983, 535, 2595, 1959, 114, 2743, 1330, 312, 0, 0, 110, 0, 319, 0, 888, 316, 1601, 2370, 2411, 3507, 2852, 2179, 58, 3151, 0, 0, 2471, 0, 826, 270, 674, 181, 0, 1555, 229, 0, 303, 211, 1875, 116, 0, 2777, 1201, 841, 975, 1466, 486, 0, 1749, 329, 109, 327, 1158, 1100, 0, 0, 940, 0, 1801, 1171, 2680, 394, 816, 1775, 2193, 731, 916, 0, 4973, 17, 1081, 28, 294, 1287, 1075, 492, 525, 1547, 1711, 327, 2641, 0, 0, 82, 707,...


In [38]:
// Retrieving just useful lines (the ones with at least a frequent singleton)
val source = Source.fromFile(data).getLines().map(x => x.split(" ").map(item => item.toInt).toSet).toSet.
              filter(line => line.intersect(L1).size > 1).toSet

source: scala.collection.immutable.Set[scala.collection.immutable.Set[Int]] = Set(Set(408, 467, 285, 499, 594, 155, 8, 346, 432), Set(814, 829, 966, 54, 236, 900, 521, 617, 854, 210), Set(217, 185, 480, 117, 678, 21, 484, 471, 339, 911, 563, 663, 927, 145, 867, 4, 897, 960, 871), Set(57, 789, 834, 307, 381, 361, 975), Set(777, 797, 966, 692, 746, 224, 67, 928), Set(352, 5, 52, 834, 736, 626, 653, 175, 832, 918, 469, 496), Set(120, 10, 598, 467, 229, 883, 145, 712, 437, 277), Set(504, 751, 78, 529, 21, 205, 932, 368, 716, 239, 860, 163, 496), Set(440, 472, 766, 798, 862, 73, 805, 844, 832), Set(606, 692, 132, 678, 647, 975, 494, 688), Set(449, 57, 948, 7, 470, 652, 55, 268), Set(52, 316, 820, 413, 831, 130, 72), Set(352, 853, 362, 567, 883, 563, 438, 978, 262, 294, 708), Set(348, 141, 88...


In [4]:
// map of all the candidates and a unique value
var possible = L1.subsets(2).toSet.zipWithIndex.toMap

possible: scala.collection.immutable.Map[scala.collection.immutable.Set[Int],Int] = Map(Set(192, 844) -> 0, Set(784, 240) -> 1, Set(995, 8) -> 2, Set(265, 579) -> 3, Set(804, 736) -> 4, Set(788, 351) -> 5, Set(265, 580) -> 6, Set(125, 995) -> 7, Set(554, 984) -> 8, Set(826, 769) -> 9, Set(766, 984) -> 10, Set(285, 701) -> 11, Set(605, 258) -> 12, Set(579, 242) -> 13, Set(521, 145) -> 14, Set(181, 649) -> 15, Set(841, 125) -> 16, Set(349, 310) -> 17, Set(458, 175) -> 18, Set(998, 351) -> 19, Set(120, 611) -> 20, Set(48, 684) -> 21, Set(57, 860) -> 22, Set(692, 678) -> 23, Set(28, 294) -> 24, Set(75, 227) -> 25, Set(217, 205) -> 26, Set(229, 991) -> 27, Set(578, 122) -> 28, Set(487, 290) -> 29, Set(38, 41) -> 30, Set(210, 496) -> 31, Set(538, 928) -> 32, Set(21, 722) -> 33, Set(819, 675) ...


In [12]:
possible(Set(217,346))

res6: Int = 48151


In [13]:
possible.filter(_._2 == 48151)

res7: scala.collection.immutable.Map[scala.collection.immutable.Set[Int],Int] = Map(Set(217, 346) -> 48151)


In [17]:
occ(possible(Set(390,227)))

res11: Int = 996


In [5]:
val occ = Array.fill(possible.keySet.size)(0)

source.foreach(line => line.subsets(2).toSet.intersect(possible.keySet).foreach(x => occ(possible(x)) = occ(possible(x)) + 1))

occ: Array[Int] = Array(47, 12, 41, 18, 13, 27, 17, 14, 19, 28, 79, 22, 16, 28, 55, 10, 15, 15, 19, 28, 41, 83, 16, 41, 9, 51, 269, 25, 16, 43, 28, 17, 30, 127, 27, 57, 31, 89, 52, 43, 22, 19, 14, 10, 39, 5, 9, 26, 7, 27, 10, 62, 28, 19, 17, 13, 3, 29, 112, 18, 44, 42, 33, 12, 21, 43, 59, 19, 96, 24, 30, 46, 74, 72, 23, 9, 17, 24, 21, 22, 17, 92, 30, 21, 12, 20, 50, 10, 32, 38, 26, 21, 91, 30, 13, 18, 188, 65, 18, 20, 62, 37, 52, 25, 15, 52, 26, 12, 37, 11, 45, 74, 193, 8, 14, 11, 14, 10, 17, 49, 12, 14, 19, 570, 9, 14, 12, 21, 27, 20, 21, 27, 58, 78, 26, 82, 41, 24, 128, 12, 21, 13, 16, 18, 6, 63, 14, 18, 17, 40, 16, 67, 12, 15, 49, 11, 35, 18, 15, 57, 21, 16, 15, 13, 27, 35, 126, 11, 25, 16, 12, 62, 37, 401, 312, 27, 16, 98, 47, 167, 5, 15, 22, 21, 39, 43, 78, 20, 18, 85, 15, 9, 6, 57...


In [6]:
val L2 = occ.zipWithIndex.filter(x => x._1 >= support).map(x => possible.filter(_._2 == x._2).map(_._1).flatten.toSet).toSet

L2: scala.collection.immutable.Set[scala.collection.immutable.Set[Int]] = Set(Set(829, 368), Set(704, 39), Set(829, 789), Set(682, 368), Set(825, 39), Set(217, 346), Set(825, 704))


In [7]:
val source = Source.fromFile(data).getLines().map(x => x.split(" ").map(item => item.toInt).toSet).toSet.
              filter(line => line.intersect(L2.flatten).size > 2).toSet

source: scala.collection.immutable.Set[scala.collection.immutable.Set[Int]] = Set(Set(666, 115, 276, 817, 825, 27, 704, 39, 858, 438, 914, 277), Set(829, 125, 789, 70, 467, 682, 281, 406, 494, 505, 690, 36, 210), Set(829, 591, 692, 522, 742, 41, 876, 368, 346, 474), Set(217, 529, 392, 33, 515, 368, 346, 716, 283, 496), Set(829, 969, 789, 489, 888, 39, 208, 516, 458, 470, 744, 368, 290, 833), Set(538, 829, 78, 147, 634, 682, 738, 919, 494, 130, 368, 854, 122), Set(825, 830, 704, 39, 859), Set(825, 156, 704, 39, 108, 175, 55, 752, 294), Set(39, 704, 782, 825), Set(25, 825, 704, 39, 703, 231, 524, 495, 699), Set(762, 829, 825, 188, 706, 96, 279, 804, 704, 145, 390, 8, 716, 867, 696, 624), Set(234, 829, 682, 513, 494, 795, 812, 414, 539, 368, 914), Set(217, 449, 110, 561, 788, 33, 682, 515,...


In [33]:
val k = 3
import scala.util.control._
val inloop = new Breaks
var pos = L2.flatten.subsets(k).toSet

    if (k > 2) {
        for (cand <- pos){
        inloop.breakable{
        for (sub: Set[Int] <- cand.subsets(k-1)) {
            if (Set(sub).intersect(L2).size == 0){
                pos -= cand
                inloop.break
        }}}
    } 
    }
val possible = pos.zipWithIndex.toMap

k: Int = 3
import scala.util.control._
inloop: scala.util.control.Breaks = scala.util.control.Breaks@40b5fb1c
pos: scala.collection.immutable.Set[scala.collection.immutable.Set[Int]] = Set(Set(825, 704, 39))
possible: scala.collection.immutable.Map[scala.collection.immutable.Set[Int],Int] = Map(Set(825, 704, 39) -> 0)


In [34]:
val occ = Array.fill(possible.keySet.size)(0)

source.foreach(line => line.subsets(3).toSet.intersect(possible.keySet).foreach(x => occ(possible(x)) = occ(possible(x)) + 1))

occ: Array[Int] = Array(1007)


In [76]:
val k = 2

count_itemsets(data, support, k, L1)

(Start ,2)
(Candidates ,70125)
(#items,7)
(Start ,3)
(Candidates ,1)
(#items,1)
(Start ,4)
(Candidates ,0)
(#items,0)


k: Int = 2
res35: Set[Set[Int]] = Set(Set(825, 704, 39), Set(829, 368), Set(704, 39), Set(829, 789), Set(682, 368), Set(825, 39), Set(217, 346), Set(825, 704))


In [72]:
def count_itemsets(data: String, support:Int, k:Int, Lk: Set[Set[Int]]): Set[Set[Int]] = {
    """
    Recursive function which searchs for frequent itemsets with k>1.
    
    Params:
    - data: path to dataset
    - support: support threshold
    - k: size of the itemset
    - candidates: possible frequent itemset
    
    Returns:
    - frequent_itemsets
    """
    println("Start ",k)
    var frequent_itemsets = Set[Set[Int]]()
    val candidates = generate_apriori(Lk, k).zipWithIndex.toMap
    println("Candidates ",candidates.size)
    val occ = Array.fill(candidates.keySet.size)(0)
    
    val source = Source.fromFile(data).getLines().map(x => x.split(" ").map(item => item.toInt).toSet).toSet.
              filter(line => line.intersect(Lk.flatten).size > k-1).toSet
    
    source.foreach(line => line.subsets(k).toSet.intersect(candidates.keySet).
                   foreach(x => occ(candidates(x)) = occ(candidates(x)) + 1))
    
    val frequent_sets_k = occ.zipWithIndex.filter(x => x._1 >= support).
            map(x => candidates.filter(_._2 == x._2).map(_._1).flatten.toSet).toSet
    
    frequent_sets_k.foreach(item => frequent_itemsets = frequent_itemsets + item)
    println("#items", frequent_itemsets.size)
    
    if (frequent_sets_k.size > 0) {
        count_itemsets(data, support, k+1, frequent_sets_k).foreach(item => frequent_itemsets = frequent_itemsets + item)
    }
    return frequent_itemsets
} 


implicit def bool2int(b:Boolean) = if (b) 1 else 0
import scala.util.control._
import scala.collection.mutable

def generate_apriori(Lk : Set[Set[Int]], k: Int): Set[Set[Int]] = {
    
    val inloop = new Breaks
    var possible = Lk.flatten.subsets(k).toSet
    
    if (k > 2) {
        for (cand <- possible){
        inloop.breakable{
        for (sub: Set[Int] <- cand.subsets(k-1)) {
            if (Set(sub).intersect(Lk).size == 0){
                possible -= cand
                inloop.break
        }}}
    } 
    }
    return possible
}

<console>:165: warning: a pure expression does nothing in statement position; multiline expressions might require enclosing parentheses
           """
           ^
count_itemsets: (data: String, support: Int, k: Int, Lk: Set[Set[Int]])Set[Set[Int]]
bool2int: (b: Boolean)Int
import scala.util.control._
import scala.collection.mutable
generate_apriori: (Lk: Set[Set[Int]], k: Int)Set[Set[Int]]


## Task 1

In [38]:
import scala.io.Source
implicit def bool2int(b:Boolean) = if (b) 1 else 0

def apriori_alg(data: String, support:Int, k:Int): List[List[Int]] = {
    
    """
    A-Priori algorithm for finding frequent itemsets with support at least 'support' in data.
    
    Params:
    - data: path to the dataset
    - support: support threshold
    - k: size of the itemset
    
    Returns: 
    - List with the frequent itemsets (by calling the function count_itemsets).
    """
    
    val source = Source.fromFile(data)
    val singletons = Array.fill(100000)(0)
    val candidates = Array.range(0, 1000) 

    for (line <- source.getLines().map(x => x.split(" ").map(item => item.toInt))) {
        for (i <- candidates){
            singletons(i) = singletons(i) + List(i).forall(line.toList.contains)
            }
        }
    source.close()

    val frequent_singl = singletons.zip(candidates).filter(x => (x._1 >= support)).map(item => item._2)
    val candidates2 = frequent_singl.toSet[Int].subsets(2).map(_.toList).toList
    
    return count_itemsets(data, support, 2, candidates2)
}
 


def count_itemsets(data: String, support:Int, k:Int, candidates: List[List[Int]]): List[List[Int]] = {
    """
    Recursive function which searchs for frequent itemsets with k>1.
    
    Params:
    - data: path to dataset
    - support: support threshold
    - k: size of the itemset
    - candidates: possible frequent itemset
    
    Returns:
    - frequent_itemsets
    """
    
    var i = 0
    val occurrences = Array.fill(candidates.size)(0)
    val frequent_itemsets = new ListBuffer[(List[List[Int]])]()
    
    println("open source",k)
    val source = Source.fromFile(data)
    for (line <- source.getLines().map(x => x.split(" ").map(item => item.toInt))) {
        if (i%1000 == 0){
        println("lines done ",i)
        }
        for (item <- candidates.zipWithIndex) {
            occurrences(item._2) = occurrences(item._2)  + item._1.forall(line.toList.contains)
        }
        i = i + 1
    }
    source.close()
    println("close source",k)
    
    val frequent_sets_k = occurrences.zip(candidates).filter(x => (x._1 >= support)).map(item => item._2).toList
    frequent_itemsets += frequent_sets_k
    
    if (frequent_sets_k.size >1) {
        val candidates2 = frequent_sets_k.flatten.toSet[Int].subsets(k+1).map(_.toList).toList
        
        frequent_itemsets += count_itemsets(data, support, k+1, candidates2)
    }
    
    return frequent_itemsets.toList.flatten
} 


import scala.io.Source
bool2int: (b: Boolean)Int
apriori_alg: (data: String, support: Int, k: Int)List[List[Int]]
count_itemsets: (data: String, support: Int, k: Int, candidates: List[List[Int]])List[List[Int]]


In [22]:
// parameters 
val data = "data/T10I4D100K.dat"
val support = 1000
val k = 1

data: String = data/T10I4D100K.dat
support: Int = 1000


In [40]:
val frequent_itmsets = apriori_alg(data, support, k)

(open source,2)
(lines done ,0)
(lines done ,1000)
(lines done ,2000)
(lines done ,3000)
(lines done ,4000)
(lines done ,5000)
(lines done ,6000)
(lines done ,7000)
(lines done ,8000)
(lines done ,9000)
(lines done ,10000)
(lines done ,11000)
(lines done ,12000)
(lines done ,13000)
(lines done ,14000)
(lines done ,15000)
(lines done ,16000)
(lines done ,17000)
(lines done ,18000)
(lines done ,19000)
(lines done ,20000)
(lines done ,21000)
(lines done ,22000)
(lines done ,23000)
(lines done ,24000)
(lines done ,25000)
(lines done ,26000)
(lines done ,27000)
(lines done ,28000)
(lines done ,29000)
(lines done ,30000)
(lines done ,31000)
(lines done ,32000)
(lines done ,33000)
(lines done ,34000)
(lines done ,35000)
(lines done ,36000)
(lines done ,37000)
(lines done ,38000)
(lines done ,39000)
(lines done ,40000)
(lines done ,41000)
(lines done ,42000)
(lines done ,43000)
(lines done ,44000)
(lines done ,45000)
(lines done ,46000)
(lines done ,47000)
(lines done ,48000)
(lines done ,4900

frequent_itmsets: List[List[Int]] = List(List(217, 346), List(829, 789), List(829, 368), List(825, 704), List(825, 39), List(682, 368), List(704, 39), List(722, 390), List(390, 227), List(825, 704, 39))


## Task 2

In [41]:
val items = frequent_itmsets.flatten.map(x => List(x)) ::: frequent_itmsets
val occurrences = Array.fill(items.size)(0)
val source = Source.fromFile(data)

for (line <- source.getLines().map(x => x.split(" ").map(item => item.toInt))) {
    for (item <- items.zipWithIndex) {
                occurrences(item._2) = occurrences(item._2)  + item._1.forall(line.toList.contains)
    }
}
source.close()
println("\nDONE")


DONE


items: List[List[Int]] = List(List(217), List(346), List(829), List(789), List(829), List(368), List(825), List(704), List(825), List(39), List(682), List(368), List(704), List(39), List(722), List(390), List(390), List(227), List(825), List(704), List(39), List(217, 346), List(829, 789), List(829, 368), List(825, 704), List(825, 39), List(682, 368), List(704, 39), List(722, 390), List(390, 227), List(825, 704, 39))
occurrences: Array[Int] = Array(5375, 3470, 6810, 4309, 6810, 7828, 3085, 1794, 3085, 4258, 4132, 7828, 1794, 4258, 5845, 2685, 2685, 1818, 3085, 1794, 4258, 1336, 1194, 1194, 1102, 1187, 1193, 1107, 1042, 1049, 1035)
source: scala.io.BufferedSource = <iterator>


In [58]:
def association_rule(itemset: List[Int], support_set: List[(List[Int], Int)], thr: Double): Int = {
    """
    Algorithm for generating association rules between frequent itemsets discovered by using the 
    A-Priori algorithm in a dataset of sales transactions.
    
    Params:
    - itemset: single frequent itemset
    - support_set: contains the support of each element of the itemsets
    - th: threshold of confidence
    
    Returns:
    - Prints the association rule with confidence > threshold.
    """
    
    //println("Itemset: $itemset")
    val left_rule = itemset.toSet[Int].subsets().map(_.toList).toList.drop(1).dropRight(1)
    val num = support_set.filter(x => x._1 == itemset).map(x => x._2: Double)
    for (i <- left_rule){
        val den = support_set.filter(x => x._1 == i).map(x => x._2: Double)
        val confidence: Double = num(0)/den(0)
        val right_rule = itemset diff i
        if (confidence>= thr) {
            println(f"{$i} -> {$right_rule}: $confidence")
        }
    }
    return 0
}

<console>:66: warning: a pure expression does nothing in statement position; multiline expressions might require enclosing parentheses
           """
           ^
association_rule: (itemset: List[Int], support_set: List[(List[Int], Int)], thr: Double)Int


In [59]:
val c = 0.5 // confidence thr
frequent_itmsets.foreach{association_rule(_, items.zip(occurrences), c)}

{List(704)} -> {List(825)}: 0.6142697881828316
{List(704)} -> {List(39)}: 0.617056856187291
{List(227)} -> {List(390)}: 0.577007700770077
{List(704)} -> {List(825, 39)}: 0.5769230769230769
{List(825, 704)} -> {List(39)}: 0.9392014519056261
{List(825, 39)} -> {List(704)}: 0.8719460825610783
{List(704, 39)} -> {List(825)}: 0.9349593495934959


c: Double = 0.5
